In [ ]:
#| default_exp core_v2

Sets the default export module for nbdev. All cells marked `#| export` will be written to `my_blog/core_v2.py`.

In [ ]:
#| hide
from nbdev.showdoc import *

Hidden cell for nbdev's `showdoc` utility—used for generating documentation from docstrings.

In [ ]:
from fastlite import Database
from pathlib import Path
from datetime import datetime, timedelta
import my_blog.config as config
from urllib.parse import quote, unquote
from fasthtml.common import *
from monsterui.all import *
from fasthtml_auth import AuthManager
from fasthtml.jupyter import *
import re
import frontmatter

Core imports: `fastlite` for SQLite database, `fasthtml` and `monsterui` for web UI, `fasthtml_auth` for authentication, and `frontmatter` for parsing markdown with YAML metadata.

In [ ]:
#| hide
import nbdev

Hidden import of nbdev for the export command at the end of the notebook.

In [ ]:
@dataclass
class AppState:
    pdb: Database # for managing posts
    posts_t: Table
    tags_t: Table
    post_tags_t: Table
    auth: AuthManager
    db: Database # For managing users and authorising access

`AppState` holds all shared application state: the posts database, table references, and auth manager. Passed around instead of using globals.

In [ ]:
def create_database_tables(pdb: Database):

    class Posts:
        id: int # primary key
        title: str
        slug: str # unique
        content: str
        created: datetime
        updated: datetime
        published: bool
        excerpt: str

    posts = pdb.create(Posts, pk='id', defaults={'published': False}, transform=True)
    posts.create_index(['slug'], unique=True, if_not_exists=True)

    class Tags:
        id: int # primary key
        name: str # unique
    
    tags = pdb.create(Tags, pk='id', transform=True)
    tags.create_index(['name'], unique=True, if_not_exists=True)

    class PostTags:
        post_id: int # foreign key > posts.id
        tag_id: int # foreign key > tags.id
    
    post_tags = pdb.create(PostTags, pk=['post_id', 'tag_id'], transform=True)

    


Creates the three database tables: `Posts` (blog content), `Tags` (category names), and `PostTags` (many-to-many junction table linking posts to tags).

In [ ]:
def create_post_database(db_path: str):
    pdb = Database(db_path)
    create_database_tables(pdb)
    return pdb

Convenience wrapper that creates a database connection and initializes all tables.

In [ ]:
def create_app():
    # Create databases and apps, return these within and AppState class.
    # Once created then create the server with srv = serve()
    # Add the routes with rt = app.route
    pdb = create_post_database(config.POSTS_DB_PATH)
    posts_t = pdb.t.posts
    tags_t = pdb.t.tags
    post_tags_t = pdb.t.post_tags
    # Initialize auth database
    auth = AuthManager(
        db_path=str(config.USERS_DB_PATH),
        config={
            'allow_registration': config.ALLOW_REGISTRATION,
            'public_paths': [],  # No public paths - all routes require auth except /auth/*
            'login_path': '/auth/login',
        }
    )
    db = auth.initialize()  
    beforeware = auth.create_beforeware()
    hdrs = (*Theme.blue.headers(highlightjs=True), Script(src="https://unpkg.com/hyperscript.org@0.9.12"),
        Link(rel="icon", type="image/png", href="/static/image/john_pixelated.png"))
    app = FastHTML(
        before=beforeware,
        secret_key=config.SECRET_KEY,
        hdrs=hdrs,
        exts='ws'  # Enable WebSocket support
    )
    config.STATIC_DIR.mkdir(parents=True, exist_ok=True)
    app.mount("/static", StaticFiles(directory=str(config.STATIC_DIR)), name="static")
    auth.register_routes(app, include_admin=True)
    state = AppState(
        pdb=pdb,
        posts_t=posts_t,
        tags_t=tags_t,
        post_tags_t=post_tags_t,
        auth=auth,
        db=db
    )
    return app, state

Main app factory: initializes both databases, sets up authentication with `fasthtml-auth`, configures headers and static files, and returns the app plus an `AppState` instance.

In [ ]:
# Initialize and run the app
app, state = create_app()
srv = JupyUvi(app)  # For notebook testing
rt = app.route

Are they the same class? True


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/cors.p

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/cors.p

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/cors.p

In [ ]:
def intro():
    return Article(
        H3("Welcome to my Blog Site", cls="text-2xl font-semibold mb-4"),
        Div(cls="text-base gap-1 text-muted-foreground leading-relaxed space-y-4")(
        P("I created this site to keep a record of things I am interested in.  As such it will largely cover motorhome trips, cycling events and routes that I have done and enjoyed, coding and software development activities I am interested in or engaged with, and technology that I think is worth looking at.  You can find out more about me on my ", hx_link("About", "/about"), " page"),

        P("This site is developed using fastHTML and the Solveit platform, both technologies developed by Jeremy Howard and ",A('Answer.ai', href='https://answer.ai', target="_blank", rel="noopener noreferrer", cls="text-primary underline"), " The desgn is based upon the site of ", A('Jack Hogan.', href='https://jackhogan.net/', target="_blank", rel="noopener noreferrer", cls="text-primary underline"),

        " See my latest blog posts below or find the full list on my ", hx_link("Blog", "/blog"), " page, where posts can be readily filtered by topic.")
        )
    )


Homepage intro section: returns an `Article` with welcome text and links to About and Blog pages using HTMX-enabled navigation.

In [ ]:
def hx_attrs(target="#main-content"): return dict(hx_target=target, hx_push_url="true", hx_swap="innerHTML show:window:top")

def hx_link(txt, href, cls="text-primary underline", target="#main-content", **kw):
    return A(txt, href=href, hx_get=href, cls=cls, **hx_attrs(target), **kw)

`hx_attrs` returns HTMX attributes for partial page updates. `hx_link` creates an anchor that uses both regular `href` and HTMX `hx-get` for SPA-like navigation.

In [ ]:
def navbar():
    brand = A(Img(src="/static/image/john_pixelated.png", alt="John Richmond", cls="w-6 h-6 rounded-full"), Span("John Richmond "), href="/", hx_get="/", cls="flex items-center gap-2 text-lg font-bold", **hx_attrs())
    links = Div(hx_link("About", "/about"), hx_link("Blog", "/blog"), cls="flex gap-4")
    return Nav(Div(brand, links, cls="flex items-center gap-2 justify-between p-4"), cls="border rounded-lg shadow bg-background")

Navigation bar with brand logo/name on the left and page links (About, Blog) on the right. Uses flexbox for layout.

In [ ]:
def x_icon(): return Svg(ft_hx("path", d="M12.6.75h2.454l-5.36 6.142L16 15.25h-4.937l-3.867-5.07-4.425 5.07H.316l5.733-6.57L0 .75h5.063l3.495 4.633L12.601.75Zm-.86 13.028h1.36L4.323 2.145H2.865z"), width=20, height=20, fill="currentColor", viewBox="0 0 16 16", aria_hidden="true")



Custom X (Twitter) icon as inline SVG—UIkit doesn't include the new X logo.

In [ ]:
def social_link(icon, href, **kw):
    kw = dict(rel="nofollow noindex") if k == "mail" else dict(target="_blank", rel="noopener noreferrer")
    return A(x_icon() if k == "twitter" else UkIcon(icon, width=20, height=20), href=href, cls="hover:text-primary transition-colors", target="_blank", rel="noopener noreferrer", **kw)

def social_link(k, v):
    ext = dict(rel="nofollow noindex") if k == "mail" else {} if k == "rss" else dict(target="_blank", rel="noopener noreferrer")
    return A(x_icon() if k == "twitter" else UkIcon(k, width=20, height=20), href=v, aria_label=k.title(), cls="hover:text-primary transition-colors", **ext)


def footer():
    links = dict(twitter="https://x.com/@johnWrichmond", youtube="https://youtube.com/@confusedjohn46a", github="https://github.com/fromLittleAcorns")
    icons = Div(*[social_link(k, v) for k, v in links.items()], social_link("mail", "mailto:confusedjohn46@gmail.com"), cls="flex justify-center gap-6 text-muted-foreground")
    return Footer(Divider(), icons, cls="max-w-2xl mx-auto px-6 mt-auto mb-6")

`social_link` creates social media icon links with appropriate `rel` attributes for security. `footer` assembles the page footer with social icons.

In [ ]:
def layout(*content, htmx, title=None):
    if htmx and htmx.request: return (Title(title), *content)
    main = Main(*content, cls='w-full max-w-2xl mx-auto px-6 py-8 space-y-8', id="main-content")
    return Title(title), Div(Div(navbar(), cls='max-w-2xl mx-auto px-4 mt-4'), main, footer(), cls="flex flex-col min-h-screen")

Page layout wrapper. On HTMX requests, returns just the content (for partial swap). On full page loads, wraps content with navbar and footer.

In [ ]:
def slug_exists(slug):
    if bool(list(state.posts_t.rows_where("slug = ?", [slug], limit=1))):
        return list(state.posts_t.rows_where("slug = ?", [slug], limit=1))[0]['id']
    else:
        return False

Checks if a slug already exists in the database. Returns the post ID if found, `False` otherwise. Used for update-vs-insert logic.

In [ ]:
from datetime import datetime

def add_post(title, content, excerpt="", tags=None, published=True):
    slug = title.lower().replace(" ", "-")
    slug = ''.join(c for c in slug if c.isalnum() or c == '-')[:60]
    posts = state.pdb.t.posts
    tags_tbl = state.pdb.t.tags
    post_tags = state.pdb.t.post_tags
    post_id = slug_exists(slug)
    if post_id:
        post = posts.update(dict(id=post_id, title=title, slug=slug, content=content, excerpt=excerpt, 
                                    created=datetime.now(), updated=datetime.now(), published=published))
    else:
        post = posts.insert(dict(title=title, slug=slug, content=content, excerpt=excerpt, 
                                    created=datetime.now(), updated=datetime.now(), published=published))
    post_id = post.id
    if tags:
        for tag in tags:
            # Get existing tags
            existing = list(tags_tbl.rows_where("name = ?", [tag], limit=1))
            # If existing tag then load the relevant id.  If not then create a new one and get the id.
            tag_id = existing[0]['id'] if existing else tags_tbl.insert(dict(name=tag))['id']
            # Check if post_tags exists for this combination and if not add
            existing = list(post_tags.rows_where("post_id= ? and tag_id= ?", [post_id, tag_id], limit=1))
            # print(existing)
            if not existing:
                # Implies no link exists for this post and tag so create one
                post_tags.insert(dict(post_id=post_id, tag_id=tag_id))
    return post_id

Creates or updates a post. Generates slug from title, handles tag creation/linking in the junction table. Updates existing posts if slug matches.

In [ ]:
@rt('/blog/{slug}')
def blogpost(htmx, slug: str):
    row = state.posts_t.rows_where("slug = ?", [slug], limit=1)
    p = next((dict(r) for r in row), None)
    if not p: return layout(H2("Not Found"), P("Post not found."), title="Not Found", htmx=htmx)
    p['created'] = datetime.fromisoformat(p['created']) if isinstance(p['created'], str) else p['created']
    content = render_md(p['content'])
    return layout(H1(p['title'], cls="text-3xl font-bold mb-2"), Span(p['created'].strftime('%B %d, %Y'), cls="text-muted-foreground text-sm mb-8 block"), content, title=p['title'], htmx=htmx)

Route to display a single blog post. Fetches by slug, parses the datetime, renders markdown content with `render_md`.

In [ ]:
@rt
def index(htmx):
    posts = get_posts(n=4)
    items = [A(H3(p['title']), P(p['excerpt'], cls="text-muted-foreground"), Span(p['created'].strftime('%d %b %Y'), cls="text-sm text-muted-foreground"), href=f"/blog/{p['slug']}", hx_get=f"/blog/{p['slug']}", cls="block border-b pb-4 hover:bg-muted/50 transition-colors", **hx_attrs()) for p in posts]
    content = Div(*items, cls="space-y-4") if items else P("No posts yet.", cls="text-muted-foreground")
    return layout((intro(), Divider(), Section(H3("Latest Posts", cls="text-xl font-semibold mb-4"), content)), title="Welcome to my Blog", htmx=htmx)

Homepage route: displays intro section, a divider, and the latest posts as cards.

In [ ]:
def get_tags(tags_tbl):
    tags = [row.name for row in tags_tbl()]
    return tags


Returns all tag names from the tags table as a list.

In [ ]:
def get_post_tags(post_id: int):
    query = """
    SELECT name FROM tags WHERE id IN (SELECT tag_id FROM post_tags WHERE post_id=?)
    """
    tag_name_dicts = state.pdb.q(query, [post_id])
    tag_names = [name['name'] for name in tag_name_dicts]
    return tag_names

Fetches all tags associated with a specific post via the `post_tags` junction table.

In [ ]:
def get_posts(n: Union[int, None]=None, tags: Union[List, None] = None):
    if tags:
        place_holders = ','.join('?'*len(tags))
        post_query = f"SELECT DISTINCT post_id FROM post_tags WHERE tag_id IN (SELECT id FROM tags WHERE name IN ({place_holders}))"
        post_ids = state.pdb.q(post_query, tags)
        post_ids = [post['post_id'] for post in post_ids]
        place_holders = ','.join('?'*len(post_ids))
        published_query = f"SELECT id FROM posts WHERE published=True AND id IN {place_holders}"
        non_draft_ids = state.pdb.q(post_query, tags)
        non_draft_ids = [post['id'] for post in non_draft_ids]
        if post_ids:
            place_holders = ','.join('?'*len(non_draft_ids))
            post_query = f"id IN ({place_holders})"
            posts = list(state.posts_t.rows_where(post_query, limit=n, order_by="-created"))
        else:
            posts = []
    else:
        posts = list(state.posts_t.rows_where("published = ?", [True], order_by="-created", limit=n))

    if posts:
        result = [dict(r) for r in posts]
        for p in result: p['created'] = datetime.fromisoformat(p['created']) if isinstance(p['created'], str) else p['created']
        for p in result: p['tags'] = get_post_tags(p['id'])
        return result
    return []

In [ ]:
tags = ['cycling', 'coding']
place_holders = ','.join('?'*len(tags))
post_query = f"SELECT DISTINCT post_id FROM post_tags WHERE tag_id IN (SELECT id FROM tags WHERE name IN ({place_holders}))"
place_holders
post_ids = state.pdb.q(post_query, tags)
print(f'post_ids: {post_ids}')
post_ids = [post['post_id'] for post in post_ids]
place_holders = ','.join('?'*len(post_ids))
published_query = f"SELECT id FROM posts WHERE published=True AND id IN ({place_holders})"
non_draft_ids = state.pdb.q(published_query, post_ids)
non_draft_ids = [post['id'] for post in non_draft_ids]
non_draft_ids

post_ids: [{'post_id': 1}, {'post_id': 2}, {'post_id': 3}, {'post_id': 4}, {'post_id': 5}]


[1, 2, 3, 4]

Main post retrieval function. Optionally filters by tags and limits results. Returns dicts with parsed datetime and tag list attached.

In [ ]:
# Example: get recent posts, optionally filtered by tags
posts = get_posts(n=5)  # Latest 5
posts = get_posts(tags=['cycling', 'motorhome'])  # Filtered by tags

In [ ]:
def tag_pill(tag_name, selected_tags):
    if tag_name in selected_tags:
        new_tags = selected_tags - {tag_name}
        selected = True
    else:
        new_tags = selected_tags | {tag_name}
        selected = False
    link = f"/blog?tags={','.join(new_tags)}" if new_tags else "/blog"
    cls = [ButtonT.primary if selected else ButtonT.secondary, ButtonT.sm, "rounded-lg"]
    return Button(tag_name, cls=cls, hx_get=link, **hx_attrs("#posts-list"))

Creates a clickable tag button. Clicking adds/removes the tag from the current filter. Selected tags are styled differently.

In [ ]:
def tag_filter(selected):
    # Return a div containing all of the tags and their selection status. We also need a button to clear the current selection
    selected: set # a set containing the names of the selected tags
    tags = get_tags(state.tags_t)
    tag_pills = [tag_pill(tag_name, selected) for tag_name in tags]
    clear_btn = Button("X Clear", cls=[ButtonT.default, ButtonT.sm, "rounded-lg"], hx_get="/blog", **hx_attrs("#posts-list"))
    return Div(P("Filter: "), *tag_pills, clear_btn, cls="flex flex-wrap gap-2 items-center", id="tag-filter")

Builds the tag filter bar: all tag pills plus a "Clear" button to reset filters.

In [ ]:
def tag_badge(name):
    return Span(name, cls="text-xs px-2 py-1 rounded bg-muted")

Small styled badge for displaying a tag name on post cards.

In [ ]:
@rt
def blog(htmx, tags:str=None):
    # selected is a SET of the name of the selected tags
    selected = {unquote(t.strip()) for t in (tags or '').split(',') if t.strip()}
    filtered = get_posts(tags=selected)
    tag_filter_div = tag_filter(selected)
    items = [post_card(p) for p in filtered]
    post_content = Div(*items, cls="space-y-2", id="posts-list") if items else P("No posts yet.", cls="text-muted-foreground", id="posts-list")
    if htmx and htmx.target == "posts-list":
        tag_filter_div.attrs['hx-swap-oob'] = 'true'
        return post_content, tag_filter_div
    return layout(H2("Blog"), tag_filter_div, Divider(cls=('my-2')), post_content, title="Blog", htmx=htmx)

Blog listing route. Parses tag filter from URL, fetches matching posts, renders tag filter + post cards. Uses HTMX OOB swap to update both filter and list on tag clicks.

In [ ]:
def get_post_image(p):
    # check post content for image (/static/image/post_images/*)
    # If image found then load a thumbnail of it
    img_ptn = r"!\[.*?\]\((/static/image/post_images/[^)]+)\)"
    imgs = re.findall(img_ptn,p["content"])
    img = imgs[0] if len(imgs)>0 else None
    if img:
        img_path = Path(img)
        img_path = config.STATIC_DIR.parent / img_path
        return img_path
    else:
        return None

Extracts the first image path from a post's markdown content for use as a thumbnail.

In [ ]:
def post_card(p):
    """Create a card to view a summary of the post including
    - Title (linked)
    - Excerpt
    - possible thumbnail image if one is in the post, to the right of the text
    - Date
    - Tags as small pills?
    - Hover effect?
    """
    img_url = get_post_image(p)
    post = Div(cls="flex gap-2 p-3 -mx-3 rounded-lg hover:bg-muted/50 hover:shadow-lg transition-all cursor-pointer")(
        Div(cls="flex-1")(
            A(cls="flex gap-4 block border-b pb-4 transition-colors")(
                Div(H3(p['title']), P(p['excerpt'], cls="text-muted-foreground"),  
                Div(Span(p['created'].strftime('%d %b %Y'), cls="text-sm text-muted-foreground"), Div([tag_badge(tag) for tag in p["tags"]], cls="flex gap-2 flex-wrap")
                )),
                Img(src=img_url, cls="p-4 max-w-48 h-auto object-contain ml-auto") if img_url else None,
                href=f"/blog/{p['slug']}",
                hx_get=f"/blog/{p['slug']}", **hx_attrs()
            )
        )
    )
    return post

Renders a post summary card with title, excerpt, date, tags, and optional thumbnail. Entire card is clickable via HTMX.

In [ ]:
def process_upload(content: bytes, filename: str, slug:str=None):
    # check there is no parent path element and if so just get the filename part
    file_path = Path(filename)
    if file_path.suffix == '.md':
        # post content, extract metadata and save contents to posts table
        md_text = content.decode('utf-8')
        post = frontmatter.loads(md_text)
        # Add checks that the meta data needed is present
        title = post.metadata['title']
        tags = post.metadata['tags']
        excerpt = post.metadata['excerpt']
        slug = title.lower().replace(" ", "-")
        slug = ''.join(c for c in slug if c.isalnum() or c == '-')[:60]
        content_rewritten = rewrite_image_paths(post.content, slug)
        post.content = content_rewritten
        try:
            add_post(title=title, content=post.content, excerpt=excerpt, tags=tags)
        except:
            # add a message that the post could not be added and the failure mode and return to the upload form
            success = False
            message = "Unable to save post"
            return success, message
        return True, "Post saved", slug

    elif file_path.suffix in ['.jpg', '.png', '.jpeg', '.tif', '.svg']:
        # file is an image, save to image folder
        path_to_save = Path(config.POST_IMAGE_DIR) / Path(slug) / file_path.name
        # Create directory if it doen't exist
        path_to_save.parent.mkdir(parents=True, exist_ok=True)
        path_to_save.write_bytes(content)
        return True, f"File {path_to_save.name} saved", slug

    else:
        # unknown file type, raise an error Toast
        return False, f"Unknown file type {filepath.suffix}", slug

Processes uploaded files. For `.md` files: parses frontmatter, rewrites image paths, saves to database. For images: saves to post-specific subfolder.

In [ ]:
@rt('/admin/upload')
def get(htmx):
    # Create file upload form for the post
    return Div(Div(A('Cancel', href='/', cls=f"{ButtonT.secondary} px-4 py-2"), Upload("Upload Button!", id='upload1', multiple=True), cls='flex gap-2'),
               Div(id='upload-message'),
               UploadZone(DivCentered(Span("Upload Zone"), UkIcon("upload")), id='upload2', accept='.md',
               hx_target='#upload-message', hx_swop='innerHTML'),
               cls='space-y-4')

Admin upload page (GET): displays a drag-and-drop upload zone for markdown files and images.

In [ ]:
@rt('/admin/upload')    
def post(upload2: list[UploadFile]):
    files = [(f.filename, f.file.read()) for f in upload2]
    md_files = [(n, c) for n, c in files if n.endswith('.md')]
    img_files = [(n, c) for n, c in files if not n.endswith('.md')]
    results = []
    slug = None
    if img_files and len(md_files)==0:
        return Div(Alert("Please upload a post (.md file) with images, or upload images separately", cls=AlertT.warning))
    for name, content in md_files:
        success, message, slug = process_upload(content, name)
        results.append((name, success, message))
    for name, content in img_files:
        success, message, _ = process_upload(content, name, slug=slug)
        results.append((name, success, message))
    header = ["Name", 'Success', 'Message']
    body = [[r[0], r[1], r[2]] for r in results]
    return Div(H2("Post upload results"), TableFromLists(header, body))

Admin upload handler (POST): processes markdown files first (to get slug), then images. Returns a results table showing success/failure for each file.

In [ ]:
def rewrite_image_paths(content: str, slug: str) -> str:
    img_ptn = r"(!\[.*?\])\(([^/)]+\.(jpg|jpeg|png|gif|svg))\)"
    replacement = rf"\1(/static/image/post_images/{slug}/\2)"
    return re.sub(img_ptn, replacement, content, flags=re.IGNORECASE)

Rewrites simple image filenames like `![](image.jpg)` to full paths like `![](/static/image/post_images/{slug}/image.jpg)`. Called at upload time.

### Add new About page

I have created an about page in markdown format using Obsidian (it is in this folder called About.md).  Please have a look using dialoguehelper tools. I would like advice as to where to put both the .md file and also the image files.  As this file is not being saved in the database I guess I will have to manually change the paths for the images etc unless there is a clean way of doing that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>view(path='.')</summary>

```json
{
  "id": "toolu_01EUZySMX1GrWxSKwYwnAUpu",
  "call": {
    "function": "view",
    "arguments": {
      "path": "."
    }
  },
  "result": "'Directory contents of /app/data/my-blog/nbs:\\n/app/data/my-blog/nbs/00_core_dup1.ipynb (212.7k)\\n/app/data/my-blog/nbs/00_core_backup_prior_to_refactor.ipynb (216.8k)\\n/app/data/my-blog/nbs/_quarto.yml (0.3k)\\n/app/data/my-blog/nbs/03_blog_v3.ipynb (35.6k)\\n/app/data/my-blog/nbs/00_core.ipynb (1422.6k)\\n/app/data/my-blog/nbs/john_and_malvenia.jpeg (1825.6k)\\n/app/data/my-blog/nbs/IMG_9929.jpeg (1953.7k)\\n/app/data/my-blog/nbs/01_blog_v2.ipynb (1383.4k)\\n/app/data/my-blog/nbs/styles.css (0.6k)\\n/app/data/my-blog/nbs/02_blog_v2.ipynb (1359.0k)\\n/app/data/my-blog/nbs/index.ipynb (3.8k)\\n/app/dat<TRUNCATED>"
}
```

</details>



<details class='tool-usage-details'>
<summary>view(path='About.md')</summary>

```json
{
  "id": "toolu_01ScpqetgTRkhxbW9dk2e3xq",
  "call": {
    "function": "view",
    "arguments": {
      "path": "About.md"
    }
  },
  "result": "\"![[john_and_malvenia.jpeg]]\\nMy name is John Richmond.  I am married with two grown up daughters and four amazing grandchildren.  We live in the UK between Leicester and Coventry.  I developed this site to enable me to keep a record of things I have done and am interested in.  As such the primary topics include travelling in our motorhome with my wife Malvenia, cycling activities, coding, the application of AI  and technology. The AI development work is my way of continuing to stay up to date with this fast developing technology.  Before retiring I worked in a major pharma company looking to <TRUNCATED>"
}
```

</details>

I can see your About.md file. It uses Obsidian's wiki-link syntax for images (`![[image.jpeg]]`) and references three images that are also in this folder:
- `john_and_malvenia.jpeg`
- `IMG_9929.jpeg`  
- `IMG_0376.jpeg`

For organizing this, I'd suggest:

1. **For the images**: Put them in `/static/image/about/` (similar to your post images structure but in a dedicated `about` folder)

2. **For the content**: You have two options:
   - **Option A**: Keep it as a file and load it at runtime (simpler, easier to edit)
   - **Option B**: Convert it to a Python function like your `intro()` function

For Option A, you'd put the `.md` file somewhere like `static/content/about.md` or keep it in the project root.

Would you like me to:
1. Write a simple function to load and render the markdown file, converting the Obsidian image syntax to standard markdown paths?
2. Or would you prefer to convert it directly to a Python function with the images hardcoded?

Either way, you'll want a helper to convert `![[filename.jpeg]]` to `![](/static/image/about/filename.jpeg)`. This is similar to your existing `rewrite_image_paths` function but for Obsidian syntax.

Which approach appeals to you?

I want to be able to edit posts and files using obsidian as the ground truth and hence to maintain the workflow I would need to functions to load and render the file as well as to conver the paths. Yes please create them

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here are the two functions:

```python
def convert_obsidian_images(content: str, image_base: str = "/static/image/about") -> str:
    """Convert Obsidian ![[image.ext]] syntax to standard markdown ![](/path/image.ext)"""
    pattern = r'!\[\[([^\]]+\.(jpg|jpeg|png|gif|svg))\]\]'
    replacement = rf'![]({image_base}/\1)'
    return re.sub(pattern, replacement, content, flags=re.IGNORECASE)

def load_md_file(path: str, image_base: str = None) -> str:
    """Load markdown file, optionally converting Obsidian image syntax"""
    content = Path(path).read_text()
    if image_base:
        content = convert_obsidian_images(content, image_base)
    return content
```

And then your `about_content` function would be:

```python
def about_content():
    md = load_md_file("static/content/about.md", image_base="/static/image/about")
    return render_md(md)
```

You'll need to:
1. Create the directories: `static/image/about/` and `static/content/`
2. Move your images to `static/image/about/`
3. Move `About.md` to `static/content/about.md`

Want me to help with the directory setup and file moves as well?

In [ ]:
def convert_obsidian_images(content: str, image_base: str = "/static/image/about") -> str:
    """Convert Obsidian ![[image.ext]] syntax to standard markdown ![](/path/image.ext)"""
    pattern = r'!\[\[([^\]]+\.(jpg|jpeg|png|gif|svg))\]\]'
    replacement = rf'![]({image_base}/\1)'
    return re.sub(pattern, replacement, content, flags=re.IGNORECASE)

def load_md_file(path: str, image_base: str = None) -> str:
    """Load markdown file, optionally converting Obsidian image syntax"""
    content = Path(path).read_text()
    if image_base:
        content = convert_obsidian_images(content, image_base)
    return content

In [ ]:
def about_content():
    md = load_md_file(config.STATIC_DIR / "content/About.md", image_base="/static/image/about")
    return render_md(md)

In [ ]:
@rt
def about(htmx):
    return layout(about_content(),title="About Me", htmx=htmx)

The link to about in my intro is not working, can you see why that is. Currently it does nothing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Looking at your `intro()` function, I can see the issue:

```python
hx_link("About", about)
```

You're passing the `about` function object instead of the URL string `"/about"`. The `hx_link` function expects a URL string for the `href` parameter.

Change it to:

```python
hx_link("About", "/about")
```

Same applies to the blog link - it should be `hx_link("Blog", "/blog")` not `hx_link("Blog", blog)`.

Great thats working now. The link to answer.ai and jack hogan are also not working, these have a href, is this because as an external site I need a different approach (ie just use a A())

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly right! Your `hx_link` function is designed for internal SPA-style navigation - it adds `hx_get` and `hx_target="#main-content"` which tries to fetch the content and swap it into your page. That doesn't work for external sites.

For external links, use a plain `A()`:

```python
A('Answer.ai', href='https://answer.ai', target="_blank", rel="noopener noreferrer")
```

The `target="_blank"` opens in a new tab, and `rel="noopener noreferrer"` is a security best practice for external links.

Ok I think the site is working ok now, and other suggestions or considerations or is now a good time to actually develop some posts and then make the site live?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great progress! A few things to consider before going live:

**Quick wins:**
- **Favicon** - browsers will request `/favicon.ico`
- **404 page** - currently shows basic "Not Found" - might want to style it nicely
- **Draft filtering** - your schema has `published` but `get_posts()` doesn't filter by it yet

**Nice to have:**
- **RSS feed** - useful for readers who use feed readers
- **Meta tags** - OpenGraph/Twitter cards for social sharing previews
- **Mobile testing** - check the layout on phones

**Operational:**
- **Database backup** - do you have a strategy for backing up your SQLite files?

None of these are blockers - many successful blogs launch without them. The most important thing is having content people want to read!

Which of these (if any) would you like to tackle, or shall we move on to writing posts?

Lets add the Favicon icon, and the draft filtering for now and come back to the rest once I have some descent content

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

**Favicon:**

Place your favicon file (e.g. `favicon.ico` or `favicon.png`) in your `static` directory, then add a route:

```python
@rt('/favicon.ico')
def favicon():
    return FileResponse(config.STATIC_DIR / 'favicon.ico')
```

Or if you prefer PNG, add it to your `hdrs` in `create_app`:
```python
Link(rel="icon", type="image/png", href="/static/favicon.png")
```

Do you have a favicon image ready, or need to create one?

---

**Draft filtering:**

In `get_posts()`, add a `published` filter. The simplest change is adding `"published = ?"` to your `rows_where` calls with `[True]` as the parameter, and optionally add an `include_drafts=False` parameter for admin views.

Would you like me to show the specific changes to `get_posts()`, or would you prefer to have a go yourself?

I have never used a favicon before, what is it and how do I create one?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

A **favicon** (short for "favorite icon") is the small icon that appears in browser tabs, bookmarks, and history next to your site name. It's typically 16x16 or 32x32 pixels.

To create one, you have a few options:

1. **Use an existing image** - Take a simple photo or logo and convert it using an online tool like [favicon.io](https://favicon.io) or [realfavicongenerator.net](https://realfavicongenerator.net)

2. **Create from scratch** - Use any image editor (even simple online ones) to make a small square image

3. **Use text/emoji** - favicon.io can generate one from text or an emoji if you don't have an image

Since you have that `john_pixelated.png` image in your navbar, that could work nicely as a favicon - it's already a small square image representing you!

Would you like to use that, or create something different?

Lets use john_pixelated

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Since you already have the image at `/static/image/john_pixelated.png`, just add this to your `hdrs` tuple in `create_app`:

```python
Link(rel="icon", type="image/png", href="/static/image/john_pixelated.png")
```

So the line becomes:
```python
hdrs = (*Theme.blue.headers(highlightjs=True), Script(src="https://unpkg.com/hyperscript.org@0.9.12"), Link(rel="icon", type="image/png", href="/static/image/john_pixelated.png"))
```

Give that a try and check if the icon appears in your browser tab!

In [ ]:
nbdev.nbdev_export()

Exports all cells marked with `#| export` to the `my_blog/core.py` module file.